In [10]:
from importlib import reload

import pandas as pd

import src.severa.base_client
import src.severa.fetch
from src.daterange import DateRange
reload(src.severa.base_client)
reload(src.severa.fetch)

span = DateRange(540)

f = src.severa.fetch.Fetcher()
async with f._client:
    df = await f.get_resource_allocations(span)

2023-05-30 17:36:45.140 | INFO     | src.severa.fetch:get_resource_allocations:142 - Starting task group...
2023-05-30 17:36:45.844 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET users [0]: Response (9) in 0.27s.
2023-05-30 17:36:46.072 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET users/eb8d73da-8c1c-7996-b085-332e777dff1a/resourceallocations/allocations [0]: Response (18) in 0.22s.
2023-05-30 17:36:46.305 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET users/0d100bc1-8376-7ac5-7bbb-055cdc20497d/resourceallocations/allocations [0]: Response (18) in 0.45s.
2023-05-30 17:36:46.369 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET users/78ddf0fe-9e39-5e5e-5096-a80409925de9/resourceallocations/allocations [0]: Response (5) in 0.52s.
2023-05-30 17:36:46.550 | SUCCESS  | src.severa.base_client:get_with_retries:123 - HTTP/1.1 GET users/5cd93092-3385-3e89-003a-9fe6d03a424d/resourceallocations/allo

In [257]:
ville = "2f365dcd-6ada-f6d8-8d73-4f393745e513"

src.severa.fetch.group_sum(["user"], df[df["user"] == ville])
#df[df["user"] == ville]

,values,user
0,2023-03-02 1.415532 2023-03-03 1.415532 ...,2f365dcd-6ada-f6d8-8d73-4f393745e513


In [11]:
df.dtypes

businessunit-user             string[python]
is_internal                             bool
value                                float64
user                          string[python]
project                       string[python]
phase                         string[python]
date                 datetime64[ns, tzutc()]
id                            string[python]
forecast-date            datetime64[ns, UTC]
dtype: object

In [30]:
from dotenv import load_dotenv

load_dotenv(r"C:\Users\vireima\tie-dashboard\.env")

import src.database
reload(src.database)

b = src.database.Base("kpi-dev", "allocations")

In [31]:
x = b.find()

2023-05-31 09:01:45.483 | INFO     | src.database:find:32 - Query '{}' resulted in 23438 results in 12.39s.


In [32]:
x.loc[:,["forecast-date","value"]]

,forecast-date,value
0,2023-02-27,0.642713
1,2023-02-28,0.642713
2,2023-03-01,0.642713
3,2023-03-02,0.642713
4,2023-03-03,0.642713
...,...,...
23433,2023-06-26,0.333333
23434,2023-06-27,0.333333
23435,2023-06-28,0.333333
23436,2023-06-29,0.333333


In [227]:
# Quick mock data
a = pd.Series(1, index=pd.date_range("2023-01-01", "2023-01-05", freq="D"))
b = pd.Series(1, index=pd.date_range("2023-01-03", "2023-01-08", freq="D"))
c = pd.Series(1, index=pd.date_range("2023-01-04", "2023-01-12", freq="D"))

df = pd.DataFrame({"user": ["A", "B", "B"], "unit": ["TIE"]*3, "values":[a,b,c]})

# Add series with datetime indices together
def combine_series(df):
    return reduce(partial(pd.Series.add,fill_value=0), df)

# This works nicely...
#ok = combine_series(df["values"])

# ...outputs a DF, indices are OK:
# 2023-01-01    1.0
# 2023-01-02    1.0
# 2023-01-03    2.0
# 2023-01-04    3.0
# ...
# 2023-01-12    1.0
# Freq: D, dtype: float64

# This 'works' in that the values are summed correctly,
# but the indices are gone now
#not_ok = df.groupby(["unit", "user"], as_index=False).agg(combine_series)

# This is seemingly a np array
#not_ok.loc["B", "values"]
# array([1., 2., 2., 2., 2., 2., 1., 1., 1., 1.])

grouping = ["unit", "user"]
grouped = df.groupby(grouping)

pd.DataFrame({"values":combine_series(v["values"])} | dict(zip(grouping, k)) for k, v in grouped)



,values,unit,user
0,2023-01-01 1 2023-01-02 1 2023-01-03 ...,TIE,A
1,2023-01-03 1.0 2023-01-04 2.0 2023-01-05...,TIE,B


In [298]:


hours_per_day = 1.5
index=pd.date_range(
    "2023-05-01",
    "2023-07-30",
    freq="D",
)
pd.DataFrame(index=index, data={"date":"2023-05-20", "user": "ville","value":hours_per_day})[]

,date,user,value
2023-05-01,2023-05-20,ville,1.5
2023-05-02,2023-05-20,ville,1.5
2023-05-03,2023-05-20,ville,1.5
2023-05-04,2023-05-20,ville,1.5
2023-05-05,2023-05-20,ville,1.5
...,...,...,...
2023-07-26,2023-05-20,ville,1.5
2023-07-27,2023-05-20,ville,1.5
2023-07-28,2023-05-20,ville,1.5
2023-07-29,2023-05-20,ville,1.5
